# <p style="text-align:center"> **GIAI ĐOẠN 4: XÂY DỰNG VÀ ĐÁNH GIÁ MÔ HÌNH** </p>


### <p style="text-align:center"> **I. IMPORT THƯ VIỆN CẦN THIẾT** </p>

In [ ]:
# Import thư viện cần thiết để thao tác dữ liệu
import pandas as pd
import numpy as np 

# Import thư viện để trực quan những kết quả tìm hiểu được:
import matplotlib.pyplot as plt

#Import thư viện hỗ trợ xây dựng mô hình
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score,KFold, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
#Lấy các biến từ pha trước vào
%store -r dfMidfielder
%store -r dfKeeper
%store -r dfForward
%store -r dfDefender

In [ ]:
#Lưu trữ các biến lại để tránh bị dính các note ở các giá trị chưa được khai báo 
dfMidfieder = dfMidfielder
dfKeeper = dfKeeper
dfForward = dfForward
dfDefender = dfDefender

### <p style="text-align:center"> **II.XỬ LÍ DỮ LIỆU ĐỂ PHÙ HỢP XÂY DỰNG MÔ HÌNH** </p>

Đầu tiên, vì tính chất của bộ dữ liệu không đồng nhất về các cột đặc điểm, ta sẽ không sử dụng dữ liệu của các thủ môn để xây dựng mô hình. Kế tiếp, vì các đặc điểm của tiền vệ, hậu vệ, tiền đạo cũng không đồng nhất; ta chỉ sử dụng những đặc tính chung của loại cầu thủ đó mà thôi.

In [ ]:
#Lấy các thuộc tính giống nhau trong 3 bộ dữ liệu của tiền vệ, tiền đạo và hậu vệ
colMidfielder = dfMidfieder.columns
colForward = dfForward.columns
colDefender = dfDefender.columns

same = [i for i in colMidfielder if i in colForward and i in colDefender]
print(same)

In [ ]:
#Loại bỏ các cột dữ liệu dư thừa
dfDefender = dfDefender[dfDefender.columns.intersection(same)]
dfForward = dfForward[dfForward.columns.intersection(same)]
dfMidfieder = dfMidfieder[dfMidfieder.columns.intersection(same)]

In [ ]:
#Thêm một cột vị trí ở cuối cùng
dfDefender['Position'] = 'DF'
dfForward['Position'] = 'FW'
dfMidfieder['Position'] = 'MF'

In [ ]:
#Gộp ba bảng lại thành một dataframe thống nhất
df = pd.concat([dfDefender, dfForward, dfMidfieder], ignore_index=True)

In [ ]:
#Kiểm tra kích cỡ của bộ dữ liệu
df.shape

In [ ]:
#Loại bỏ cột tên (không cần thiết cho việc xây dựng mô hình)
df.drop(columns = 'Player name', inplace=True)

Đưa dữ liệu về dạng trung bình mỗi trận đấu để khoảng cách giữa dữ liệu của các cầu thủ mới chơi
và các cầu thủ lâu năm không bị quá lệch nhau

In [ ]:
#Cầu thủ chưa ra sân nhưng lại có dữ liệu thi đấu (lỗi của data)
df.drop([723], inplace = True)


In [ ]:
y = df['Position']
y.replace({'FW': 1, 'MF': 2, 'DF': 3}, inplace = True)
df.drop(columns='Position', inplace=True)

In [ ]:
new_df = df.iloc[:,1:].apply(lambda x: round(x / df.Appearances, 2))
new_df.dtypes

In [ ]:
#FILL NA BY MEAN
new_df.fillna(round(new_df.mean(), 2), inplace=True)
new_df
    

## PHÂN CHIA TẬP TRAIN VÀ TẬP TEST

KÍCH THƯỚC TẬP TEST LÀ 20%, random state tùy ý

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.3, random_state=123)

### MÔ HÌNH LOGISTIC REGRESSION

In [281]:
param_grid = {'loss': ['hinge', 'huber', 'squared_hinge', 'perceptron'], 
            'penalty' : ['l1', 'l2', 'elasticnet'],
            'shuffle' :[True,False], 
            'fit_intercept': [True, False],
            'epsilon': [0.1, 0.5]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

sgd_grid = GridSearchCV( SGDClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='neg_mean_squared_error', 
                          verbose=3) 
sgd_grid.fit(X_train,y_train)

[CV 17/50] END epsilon=0.5, fit_intercept=True, loss=squared_hinge, penalty=l1, shuffle=False;, score=-0.819 total time=   0.0s
[CV 18/50] END epsilon=0.5, fit_intercept=True, loss=squared_hinge, penalty=l1, shuffle=False;, score=-0.661 total time=   0.0s
[CV 19/50] END epsilon=0.5, fit_intercept=True, loss=squared_hinge, penalty=l1, shuffle=False;, score=-0.543 total time=   0.0s
[CV 20/50] END epsilon=0.5, fit_intercept=True, loss=squared_hinge, penalty=l1, shuffle=False;, score=-0.468 total time=   0.0s
[CV 21/50] END epsilon=0.5, fit_intercept=True, loss=squared_hinge, penalty=l1, shuffle=False;, score=-0.654 total time=   0.0s
[CV 22/50] END epsilon=0.5, fit_intercept=True, loss=squared_hinge, penalty=l1, shuffle=False;, score=-0.504 total time=   0.0s
[CV 23/50] END epsilon=0.5, fit_intercept=True, loss=squared_hinge, penalty=l1, shuffle=False;, score=-0.528 total time=   0.0s
[CV 24/50] END epsilon=0.5, fit_intercept=True, loss=squared_hinge, penalty=l1, shuffle=False;, score=-0

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=None),
             estimator=SGDClassifier(),
             param_grid={'epsilon': [0.1, 0.5], 'fit_intercept': [True, False],
                         'loss': ['hinge', 'huber', 'squared_hinge',
                                  'perceptron'],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'shuffle': [True, False]},
             scoring='neg_mean_squared_error', verbose=3)

In [282]:
print('Best Score: %s' % sgd_grid.best_score_)
print('Best Hyperparameters: %s' % sgd_grid.best_params_)

Best Score: -0.5540357455318085
Best Hyperparameters: {'epsilon': 0.1, 'fit_intercept': True, 'loss': 'perceptron', 'penalty': 'l1', 'shuffle': True}


In [283]:
print(classification_report(sgd_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.49      0.70      0.57        33
           2       0.91      0.56      0.70       189
           3       0.45      0.96      0.61        51

    accuracy                           0.65       273
   macro avg       0.62      0.74      0.63       273
weighted avg       0.77      0.65      0.66       273



In [ ]:
# param_grid = {'solver': ['lbfgd', 'newton-cg', 'saga'], 
#             'penalty' : ['l1', 'l2', 'elasticnet'],
#             'dual' :[True,False], 
#             'fit_intercept': [True, False],
#             'C': [0.1, 0.5, 1]}
            
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

# logistic_grid = GridSearchCV( LogisticRegression(),param_grid = param_grid,
#                           cv=cv, 
#                           scoring='neg_mean_squared_error', 
#                           verbose=3) 
# logistic_grid .fit(X_train,y_train)